# Test notebook for Sx operations.

In [1]:
import servicex
from servicex import ServiceXDataset
from func_adl import EventDataset
from func_adl_xAOD import ServiceXDatasetSource
import func_adl_uproot
import uproot_methods
from numpy import genfromtxt
import qastle
import csv

### Datasets:

localds://mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00

user.emmat:user.emmat.mc16_13TeV.311311.MadGraphPythia8EvtGen_A14NNPDF31LO_HSS_LLP_mH125_mS15.mc16d.200131.forsX_trees.root

In [7]:
def test_retrieve_simple_jet_pts():
    query = "(call ResultTTree (call Select (call SelectMany (call EventDataset (list 'localds:bogus')) (lambda (list e) (call (attr e 'Jets') 'AntiKt4EMTopoJets'))) (lambda (list j) (/ (call (attr j 'pt')) 1000.0))) (list 'JetPt') 'analysis' 'junk.root')"
    dataset = ServiceXDataset("mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00")
    r = dataset.get_data_pandas_df(query)
    
    return r

In [9]:
data_xaod = test_retrieve_simple_jet_pts()

In [10]:
print(data_xaod)

            JetPt
entry            
0       52.898621
1       28.133312
2       15.308400
3        5.986582
4        5.813203
...           ...
857133   6.211655
857134  47.653145
857135  32.738951
857136   6.260789
857137   5.394783

[11355980 rows x 1 columns]


In [32]:
def test_old_simple_jet_pts():
    dataset = ServiceXDataset("mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00")
    query = ServiceXDatasetSource(dataset) \
        .SelectMany('lambda e: (e.Jets("AntiKt4EMTopoJets"))') \
        .Where('lambda j: (j.pt()/1000)>30') \
        .Select('lambda j: (j.pt())') \
        .AsPandasDF("JetPt") \
        .value()
    return query

def test_retrieve_lepton_data():
    dataset = ServiceXDataset("mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00")
    query = ServiceXDatasetSource(dataset)	\
        .Select('lambda e: (e.Electrons("Electrons"))') \
        .Select('lambda e: (e.pt(), \
                            e.eta(), \
							e.phi(), \
							e.e())') \
        .AsAwkwardArray(('ElePt', 'EleEta', 'ElePhi', 'EleE')) \
        .value()
    return query

In [33]:
lepton_test = test_retrieve_lepton_data()

ServiceXException: (ServiceXException(...), 'Failed to transform all files')

In [29]:
print(lepton_test)

{b'ElePt': <JaggedArray [[56933.7265625 30103.556640625 7957.18994140625] [8510.5205078125 4297.3701171875 3224.6416015625] [31541.15234375] ... [4459.81787109375] [52486.60546875 23942.8203125 19794.88671875] []] at 0x0221309b15c8>, b'EleEta': <JaggedArray [[-0.12912268936634064 0.5888139605522156 1.2514770030975342] [0.675038754940033 0.2761063873767853 0.12661254405975342] [-2.275493621826172] ... [0.18539121747016907] [-0.1793559193611145 1.324408769607544 -0.062219955027103424] []] at 0x0221309b1e48>, b'ElePhi': <JaggedArray [[2.520068883895874 -0.45167723298072815 -0.7712351083755493] [0.8112514615058899 -2.0128047466278076 -1.8641725778579712] [2.1214284896850586] ... [2.7884395122528076] [1.3793280124664307 -2.747081756591797 -1.044033169746399] []] at 0x022130a3f088>, b'EleE': <JaggedArray [[57409.0049476159 35474.56113861621 15045.390911737933] [10524.304487067426 4462.2178674942725 3250.5228782492427] [155111.00184215332] ... [4536.679420621884] [53333.07986649214 48195.9617

In [2]:
def test_func_adl_simple_jet_pts():
    dataset = ServiceXDataset("mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00", image="sslhep/servicex_func_adl_xaod_transformer:pr_atlas_rpm_bug_fix")
    query = ServiceXDatasetSource(dataset) \
        .Where('lambda e: e.Jets("AntiKt4EMTopoJets").Where(lambda j: j.pt() / 1000.0 > 30.0).Count() > 1') \
        .Select('lambda e: e.Electrons("Electrons")') \
        .Select('lambda e: e.Select(lambda ele: ele.eta())') \
        .AsAwkwardArray('EleEta') \
        .value()
    return query

#        .Select('lambda ls: (ls[1].Select(lambda e: e.eta()).Where(ls[0].Select(lambda j: j.pt()).Count() > 2))') \



In [3]:
data = test_func_adl_simple_jet_pts()

In [47]:
print(data)

{b'EleEta': <JaggedArray [[-0.12912268936634064 0.5888139605522156 1.2514770030975342] [0.675038754940033 0.2761063873767853 0.12661254405975342] [-2.275493621826172] ... [0.18539121747016907] [-0.1793559193611145 1.324408769607544 -0.062219955027103424] []] at 0x0221265d3708>}


In [5]:
def lambda_capture():
    data = ServiceXDataset('mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00')
    query = ServiceXDatasetSource(data) \
        .Select('lambda e: (e.Jets("AntiKt4EMTopoSets"), e.Electrons("Electrons"))') \
        .Select('lambda ls: (ls[0].Select(lambda j: j.pt()), \
                             ls[0].Select(lambda j: j.eta()), \
                             ls[0].Select(lambda j: j.phi()), \
                             ls[1].Select(lambda e: e.pt()), \
                             ls[1].Select(lambda e: e.eta()), \
                             ls[1].Select(lambda e: e.phi()))') \
        .Where('lambda ls: abs(sqrt((ls[0].Select(lambda j: j.eta()))**2 + (ls[0].Select(lambda j: j.phi()))**2) \
               - sqrt((ls[1].Select(lambda e: e.eta()))**2 + (ls[1].Select(lambda e: e.phi()))**2)) < 1.0') \
        .Select('lambda ls: (ls[0].Select(lambda j: j.pt()), ls[1].Select(lambda e: e.pt()))') \
        .AsAwkwardArray(("JetPt", "ElePt")) \
        .value()
    return query

In [6]:
lambda_test = lambda_capture()

ServiceXException: (ServiceXException(...), 'ServiceX rejected the transformation request: (400){\'message\': "Failed to submit transform request: Failed to generate translation code: No terminal defined for \'*\' at line 1 col 778\\n\\nist ls) (< (call abs (- (call sqrt (+ (** (call (attr (subscript ls 0) \'Select\')\\n                                        ^\\n\\nExpecting: {\'WHITESPACE\', \'RPAR\'}\\n"}')

In [2]:
def test_servicex_stress_capacity():
    query = EventDataset('localds://data17_13TeV:data17_13TeV.periodK.physics_Main.PhysCont.DAOD_STDM7.grp22_v01_p3713') \
        .SelectMany('lambda e: (e.Jets("AntiKt4EMTopoJets"))') \
        .Select('lambda j: (j.pt())') \
        .AsPandasDF("JetPt") \
        .value(executor=lambda a: use_exe_servicex(a, endpoint='http://localhost:5000/servicex')) \
        .MaxWorkers(100)
    
    return query

In [ ]:
stress_test = test_servicex_stress_capacity()